# Feature Engineering

In [5]:
import numpy as np
import pandas as pd

In [6]:
# Once data cleaning is complete, we'll switch to reading in the cleaned version. 
# For now, using the raw data to verify that the code functions as expected.
df = pd.read_csv('LifeExpectancyDraft.csv')

Population Density

In [7]:
# Load Area Data
area_df = pd.read_csv('LandAreakm.csv')

# Rename Columns for clarity
area_df = area_df.rename(columns={'Entity': 'Country', 'Land area (sq. km)': 'Area'})

# Add 'Area' Column to df
area_df = area_df.groupby('Country', as_index=False).agg({'Area': 'first'})
df = df.merge(area_df[['Country', 'Area']], on='Country', how='left')

# Drop Duplicate Rows
df = df.drop_duplicates()

# Create Population Density Column
df['PopulationDensity'] = df['Population'] / df['Area']

# Remove Area Column
df = df.drop('Area', axis=1)

BMI Classification

In [8]:
# Create columns for each BMI classification based on the BMI range
df['UnderweightBMI'] = (df[' BMI '] < 18.5).astype(int)
df['HealthyWeightBMI'] = ((df[' BMI '] >= 18.5) & (df[' BMI '] < 25)).astype(int)
df['OverweightBMI'] = ((df[' BMI '] >= 25) & (df[' BMI '] < 30)).astype(int)
df['ObesityBMI'] = (df[' BMI '] >= 30).astype(int)

KeyError: ' BMI '

Income Classification

In [ ]:
# Low-income economies: GNI per capita of $1,135 or less
# Lower-middle-income economies: GNI per capita between $1,136 and $4,465
# Upper-middle-income economies: GNI per capita between $4,466 and $13,845
# High-income economies: GNI per capita of $13,846 or more
df['LowIncome'] = (df['GDP'] < 1136).astype(int)
df['LowerMiddleIncome'] = ((df['GDP'] >= 1136) & (df['GDP'] < 4466)).astype(int)
df['UpperMiddleIncome'] = ((df['GDP'] >= 4466) & (df['GDP'] < 13846)).astype(int)
df['HighIncome'] = (df['GDP'] >= 13846).astype(int)

Health Coverage Index 
- Positive index means above average 
- Negative index means below average
- Zero index means exactly average

In [ ]:
# Reverse necessary columns so a higher number means better health
for col in ['Measles', 'HIV/AIDS', 'Thinness 1-19 Years', 'Thinness 5-9 Years']:
    df[f'{col}_adjusted'] = df[col].max() - df[col]

# Standardize all columns using z-score normalization
for col in ['Hepatitis B', 'Polio', 'Diphtheria', 'Measles_adjusted', 'HIV/AIDS_adjusted', 'Thinness 1-19 Years_adjusted', 'Thinness 5-9 Years_adjusted']:
    df[f'{col}_z'] = (df[col] - df[col].mean()) / df[col].std()

# Combine into a health index by taking the weighted mean of z-scores
z_cols = [f'{col}_z' for col in ['Hepatitis B', 'Polio', 'Diphtheria', 'Measles_adjusted', 'HIV/AIDS_adjusted', 'Thinness 1-19 Years_adjusted', 'Thinness 5-9 Years_adjusted']]
weights = {
    'Hepatitis B_z': 0.2,
    'Polio_z': 0.2,
    'Diphtheria_z': 0.2,
    'Measles_adjusted_z': 0.15,
    'HIV/AIDS_adjusted_z': 0.1,
    'Thinness 1-19 Years_adjusted_z': 0.075,
    'Thinness 5-9 Years_adjusted_z': 0.075,
}

# Normalize weights 
total_weight = sum(weights.values())
weights = {k: v / total_weight for k, v in weights.items()}

# Calculate the health index (handles missing data)
df['Health Coverage Index'] = df[z_cols].apply(
    lambda row: np.average(
        row.dropna(), 
        weights=[weights[col] for col in row.index if col in weights]
    ) if not row.isnull().all() else np.nan,
    axis=1
)

In [ ]:
df.to_csv('LifeExpectancyClean.csv')